In [1]:
from transformers import pipeline
import librosa
import sounddevice as sd

In [2]:
model_pth = r'D:\model_code\server_models\wav2vec2\trial_2\wav2vec2-finetune'

In [3]:
pipe = pipeline("audio-classification", model=model_pth)

Some weights of the model checkpoint at D:\model_code\server_models\wav2vec2\trial_2\wav2vec2-finetune were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at D:\model_code\server_models\wav2vec2\trial_2\wav2vec2-finetune and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.p

In [4]:
def get_label(x):
    y = None
    mx_n = -1
    for i in x:
        if i['score'] > mx_n:
            mx_n = i['score']
            y = i['label']
    return y, mx_n


Inference

In [5]:
fs = 44100
sr = 16000
buffer_duration = 2 # seconds
buffer_size = buffer_duration * fs

In [33]:
while True:
    try:
        # print("Recording...")
        # Record audio for 2 seconds
        buffer = sd.rec(buffer_size, channels=1, dtype='float32')
        sd.wait()  # Wait until recording is finished
        sd.play(buffer,fs)
        sd.wait()  # Wait until recording is finished
        # Process the recorded audio
        y = buffer.squeeze()[fs:]
        data = librosa.resample(y=y,orig_sr=fs,target_sr=sr)
        l = get_label(pipe(data))
        print(l or '',end=' ')
        # break
        # clear_output(wait=True)
    except KeyboardInterrupt:
        print("Stream stopped by user")
        break

  adele adele     hilfe hilfe adele      adele      adele  Stream stopped by user


In [6]:
import librosa
y,sr = librosa.load(r'D:\model_code\web_app\static\audios\2024-07-16_20-24\unknown\58_unknown.wav',sr=None)
pipe(y),sr

([{'score': 0.5576029419898987, 'label': 'unknown'},
  {'score': 0.391681432723999, 'label': 'hilfe'},
  {'score': 0.05071566626429558, 'label': 'adele'}],
 16000)